## Library Data Analysis

Data analysis covered in this notebook aims to prove the answers and solutions to the business problems libraries face, books being checked out and returned late. 

The analysis will discover connections between entities engaged in this business transaction, customer, books, and libraries, by looking into their respective relationships. 
This part of the process is called EDA (Exploratory Data Analysis) and if successful, it is going to yield features that are the most descriptive of the problem and these features will help us build a predictive model.

The main idea is that the model will be able to help libraries elevate their businesses. 

This notebook
1. Loads the data
2. (Potentially) Adds additional columns to existing data (providing insight)
3. Detects and visualizes the features
4. Explains how these features can help mitigate the risks libraries are facing

In [1]:
from datetime import datetime, timezone
from pathlib import Path

import hvplot.polars
import polars as pl

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [2]:
RETURN_LIMIT = 28

### Paths and tables

Defines paths and load tables showcasing some data points.

In [3]:
ROOT_PATH = Path(__name__).absolute().parent.parent
DATA_PATH = ROOT_PATH / "data"

TABLE_PATH = DATA_PATH / "storage" / "silver"

The available datasets are small, so we can load them into memory with the `read_parquet` function. If circumstances were different, we would use `LazyFrame` with lazy evaluation (with better optimizations for operations down the road).

Data is loaded from the "silver" layer or the layer that has clean data ready for the analysis. 

### Books table

In [4]:
# books

books_path = TABLE_PATH / "books.parquet"
books_df = pl.read_parquet(source=books_path)

books_df.head(1)

id,title,authors,publisher,published_date,categories,price,pages,published_year
str,str,list[str],str,str,list[str],f32,i32,i32
"""_C3JCwAAQBAJ""","""Engines Of Anxiety""","[""Wendy Nelson Espeland""]","""Russell Sage Foundation""","""2016-05-09""","[""Social Science""]",39.0,382,2016


In [5]:
print("Nulls in books columns:\n")

book_nulls = books_df.select(pl.all().is_null().sum()).to_dicts()[0]
print(book_nulls)

Nulls in books columns:

{'id': 0, 'title': 0, 'authors': 73, 'publisher': 146, 'published_date': 2, 'categories': 51, 'price': 2, 'pages': 0, 'published_year': 2}


### Customers table

In [6]:
# customers

customers_path = TABLE_PATH / "customers.parquet"
customers_df = pl.read_parquet(source=customers_path)

customers_df.head(1)

id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation
str,str,str,str,str,i32,date,str,str,str
"""564a19483a1b1379899f9d89e4b286…","""Carol Stanfill""","""2926 Ne 9th Ave""","""Portland""","""Oregon""",97212,1960-04-22,"""female""","""College""","""Others"""


In [7]:
print(f"Number of customers: {len(customers_df)}")

Number of customers: 2000


In [8]:
print("Nulls in customer columns:\n")

customer_nulls = customers_df.select(pl.all().is_null().sum()).to_dicts()[0]
print(customer_nulls)

Nulls in customer columns:

{'id': 0, 'name': 0, 'street_address': 0, 'city': 94, 'state': 102, 'zipcode': 1, 'birth_date': 107, 'gender': 101, 'education': 104, 'occupation': 107}


10% of the customers have null values for some of their attributes. 

One way to solve the issue is to drop rows with null values and the other is to try and mitigate the problem by replacing null values with median values. A good candidate is the `age` column. Other is `gender` where we can guess the gender but that's error-prone.

In [9]:
# Fill null values
books_df = books_df.with_columns(
    pl.col("price").fill_null(pl.col("price").median()).alias("price")
)

### Libraries table

In [10]:
# libraries

libraries_path = TABLE_PATH / "libraries.parquet"
libraries_df = pl.read_parquet(source=libraries_path)

libraries_df.head(1)

id,name,street_address,city,region,postal_code
str,str,str,str,str,str
"""227-222@5xc-jww-btv""","""Multnomah County Library Hills…","""1525 Sw Sunset Blvd""","""Portland""","""OR""","""97239"""


### Checkouts table

In [11]:
# checkouts

checkouts_path = TABLE_PATH / "checkouts.parquet"
checkouts_df = pl.read_parquet(source=checkouts_path)

checkouts_df.head(1)

id,patron_id,library_id,date_checkout,date_returned
str,str,str,date,date
"""1Mc0AQAAMAAJ""","""f073be688f2016dc22f318b7113e06…","""226-222@5xc-jxj-7yv""",2018-05-09,2018-05-24


### Taking a look at the tables

Looking at the tables, there are candidates for some additional columns that may help with the analysis and are potential candidates for features. 
For example, the `customers` table has the field `birth_date`. Creating age categories (Young, Adults, Middle-Aged Adults, and Old) can give additional insight into who rents books.

Also, the `checkouts` table has columns `date_checkout` and `date_returned,` which is going to be our label.

In [12]:
customers_df = customers_df.with_columns(
    ((datetime.now(timezone.utc).date() - pl.col("birth_date")).dt.total_days() / 365)
    .floor()
    .cast(pl.Int32)
    .alias("age")
)

In [13]:
customers_df = customers_df.with_columns(
    pl.when((3 <= pl.col("age")) & (pl.col("age") <= 19))
    .then(pl.lit("Young"))
    .when((20 <= pl.col("age")) & (pl.col("age") <= 39))
    .then(pl.lit("Adult"))
    .when((40 <= pl.col("age")) & (pl.col("age") <= 59))
    .then(pl.lit("Middle-Age"))
    .when((60 <= pl.col("age")) & (pl.col("age") <= 99))
    .then(pl.lit("Old"))
    .otherwise(pl.lit("Undefined"))
    .alias("age_category")
)

In [14]:
customers_df.head(3)["age", "age_category"]

age,age_category
i32,str
64,"""Old"""
71,"""Old"""
19,"""Young"""


In [15]:
# Fill null values
customers_df = customers_df.with_columns(
    pl.col("age").fill_null(pl.col("age").median()).alias("age")
)

### Label calculation

Labels can be calculated by subsctracting `date_returned` and `date_checkout`. Limit for the book return is 28 days. 

Return date has to be higher than checkout date. If return date is `None` that means customer didn't returned book yet.

In [16]:
checkouts_df = checkouts_df.with_columns(
    pl.col("date_returned")
    .sub(pl.col("date_checkout"))
    .dt.total_days()
    .cast(pl.Int32)
    .alias("days_holding_book")
)

checkouts_df = checkouts_df.with_columns(
    pl.when(
        (pl.col("days_holding_book") <= RETURN_LIMIT)
        & (pl.col("days_holding_book") >= 0)
    )
    .then(pl.lit(1))
    .when(pl.col("days_holding_book") > RETURN_LIMIT)
    .then(pl.lit(0))
    .when(pl.col("date_returned").is_null())
    .then(pl.lit(0))
    .when(pl.col("days_holding_book") < 0)
    .then(pl.lit(-1))
    .otherwise(pl.lit(-1))
    .alias("book_returned")
)

If there are values that are negative values in column `days_holding_book` number `-1` marks them as unvalid. 

In [17]:
checkouts_df.head(2)

id,patron_id,library_id,date_checkout,date_returned,days_holding_book,book_returned
str,str,str,date,date,i32,i32
"""1Mc0AQAAMAAJ""","""f073be688f2016dc22f318b7113e06…","""226-222@5xc-jxj-7yv""",2018-05-09,2018-05-24,15,1
"""sBmIDwAAQBAJ""","""bad6c410ae2a8b7a069f4f6398f576…","""226-222@5xc-jxj-7yv""",2018-07-06,2018-08-01,26,1


### Entity analysis
Analysis of customers and book entities
TODO: Add more description

In [18]:
gender_df = customers_df.group_by(pl.col("gender")).len()
gender_df = gender_df.drop_nulls().rename(
    {"gender": "Gender", "len": "Number of customers"}
)
gender_df.hvplot.bar(
    x="Gender", y="Number of customers", rot=90, title="Library customers gender"
)

:Bars   [Gender]   (Number of customers)

In [19]:
education_df = customers_df.group_by(pl.col("education")).len()
education_df = education_df.drop_nulls().rename(
    {"education": "Education", "len": "Number of customers"}
)
education_df.hvplot.bar(
    x="Education",
    y="Number of customers",
    rot=90,
    color="teal",
    title="Library customer education",
)

:Bars   [Education]   (Number of customers)

In [20]:
occupation_df = customers_df.group_by(pl.col("occupation")).len()
occupation_df = occupation_df.drop_nulls().rename(
    {"occupation": "Occupation", "len": "Number of customers"}
)
occupation_df.hvplot.bar(
    x="Occupation",
    y="Number of customers",
    rot=90,
    color="orange",
    title="Library customer occupation",
)

:Bars   [Occupation]   (Number of customers)

In [21]:
age_df = customers_df.group_by(pl.col("age_category")).len()
age_df = age_df.drop_nulls().rename(
    {"age_category": "Age Category", "len": "Number of customers"}
)
age_df = age_df.filter(pl.col("Age Category") != "Undefined")
age_df.hvplot.bar(
    x="Age Category",
    y="Number of customers",
    rot=90,
    color="purple",
    title="Library customer age group",
)

:Bars   [Age Category]   (Number of customers)

By analyzing categorical columns of customers we can conclude that:
1) Looking at the gender there are almost even number of male and female customers. 10% of the data are `nulls` or unknown values.
2) Education seems to be equally distributed among customers
3) The same applies to the occupation
4) By looking at the age group there are dominantly adult renters while young are around 10%.

It can be interesting to see which of these groups are late to return the books or if there is even a connection.

In [22]:
books_df.hvplot.hist(
    y="pages", bins=50, bin_range=(-50, 1100), title="Book pages distribution"
)

:Histogram   [pages]   (Count)

In [23]:
books_df.hvplot.hist(
    y="price", bins=50, bin_range=(-150, 800), title="Book prices distribution"
)

:Histogram   [price]   (Count)

In [24]:
book_categories_df = books_df.group_by(pl.col("categories")).len()
book_categories_df = book_categories_df.drop_nulls()
book_categories_df.hvplot.bar(title="Book categories distribution")

:Bars   [index]   (len)

There are three dominant categories of books while the rest are below 10 unit per category(ies).

In [25]:
books_df.hvplot.hist(
    y="published_year", bins=30, title="Book publishing year distribution"
)

:Histogram   [published_year]   (Count)

Number of books published in the same year might not be relevent but we can what is rented the most (taking publishing year in account) and connect it to the customer data (age group, education etc.)

In [26]:
book_returned_df = checkouts_df.group_by("book_returned").len()
book_returned_df = book_returned_df.drop_nulls().rename(
    {"book_returned": "Book returned", "len": "Number of customers"}
)
book_returned_df.hvplot.bar(
    x="Book returned",
    y="Number of customers",
    rot=90,
    stacked=True,
    color="purple",
    title="Customers returning/not returning books",
)

:Bars   [Book returned]   (Number of customers)

By analyzing data about books we see that: 
1. Number of book categories and their distribution in total number of books is not relevant, it's doesn't tell much. There are a couple of categories that are dominant.
2. From publishing years we can see deficiency of books published between 1940. and 1960. year. These informations are not relevant for the problem we are trying to solve
3. Both prices and number of pages are following normal distribution
4. 20% does not return rented books on time
5. 10% of rented books data is invalid

Adding publisher column to the list of analyzed data doesn't accomplish anything and this column is not valuable as potential feature. 
Already from the `Customers returning/not returning books` bar plot we can see imbalance in number of positive and negative labels. Training predictive model is going to be harder. 

### Connecting data together

Joining customers and books data through checkouts will tell what type of customers and what types of books are returning late.
Selected columns should be tested for correlation with the label. 
After correlation check, columns above the threshold can be used for training predictive model.

In [27]:
customer_checkout_df = customers_df.rename({"id": "patron_id"}).join(
    checkouts_df, on="patron_id"
)
library_df = customer_checkout_df.join(books_df, on="id", how="left")

print(f"Number of data points: {len(library_df)}")

Number of data points: 2000


Number of data points fits the number of customers. 

### Columns and the resulting table for analysis

In [28]:
library_df.head(3)

patron_id,name,street_address,city,state,zipcode,birth_date,gender,education,occupation,age,age_category,id,library_id,date_checkout,date_returned,days_holding_book,book_returned,title,authors,publisher,published_date,categories,price,pages,published_year
str,str,str,str,str,i32,date,str,str,str,f64,str,str,str,date,date,i32,i32,str,list[str],str,str,list[str],f32,i32,i32
"""f073be688f2016dc22f318b7113e06…","""Jill Hamood""","""3036 Ne Everett St""","""Portland""","""Oregon""",97232,1814-10-13,"""female""",null,"""Blue Collar""",210.0,"""Undefined""","""1Mc0AQAAMAAJ""","""226-222@5xc-jxj-7yv""",2018-05-09,2018-05-24,15,1,"""The Institutes Of Medicine""","[""Martyn Paine""]",null,"""1862""","[""Medicine""]",119.0,600,1862
"""bad6c410ae2a8b7a069f4f6398f576…","""Richard Rangaswamy""","""10344 Ne Skidmore St""","""Portland""","""Oregon""",97220,2119-08-14,null,"""Others""","""Education & Health""",-95.0,"""Undefined""","""sBmIDwAAQBAJ""","""226-222@5xc-jxj-7yv""",2018-07-06,2018-08-01,26,1,"""Automotive Engine Performance""","[""Nicholas Goodnight""]","""Jones & Bartlett Learning""","""2019-02-22""","[""Transportation""]",196.0,703,2019
"""20aea0583cd89cdcb437bf2eb40a38…","""Robert Crespo""","""90 Nw Davis St""","""Portland""","""Oregon""",97209,null,"""male""","""Graduate Degree""","""Others""",45.0,"""Undefined""","""teQaAQAAMAAJ""","""222-222@5xc-jv5-nt9""",null,2018-04-09,null,-1,"""Directory Of Northern Library …",null,null,"""1965""","[""Libraries""]",101.0,372,1965


In the table above, we have sample data and all columns listed. This mini table represent joined data between customers, books and checkouts.
By analyzing this dataframe one can check which customers returned books (or not), what kinds of books, their prices, pages, categories etc.
We can add also information about libraries and check which library has to most problems about late returnings.

In [29]:
sub_library_df = library_df.select(
    [
        "patron_id",
        "name",
        "gender",
        "education",
        "occupation",
        "age",
        "age_category",
        "days_holding_book",
        "title",
        "published_year",
        "categories",
        "price",
        "pages",
        "book_returned",
    ]
)
sub_library_df.hvplot.table(
    columns=["name", "gender", "age", "title", "price", "book_returned"],
    sortable=True,
    selectable=True,
)

:Table   [name,gender,age,title,price,book_returned]

In [30]:
customer_agg_df = sub_library_df.group_by("gender", "book_returned").agg(
    gender_returning_book=pl.count("gender"),
    median_reader_age=pl.median("age"),
    median_book_price=pl.median("price"),
    median_days_renting=pl.median("days_holding_book"),
    mean_book_pages=pl.mean("pages"),
)
customer_agg_df = customer_agg_df.drop_nulls()
customer_agg_df

gender,book_returned,gender_returning_book,median_reader_age,median_book_price,median_days_renting,mean_book_pages
str,i32,u32,f64,f32,f64,f64
"""female""",-1,158,46.5,286.0,-32866.5,572.506329
"""male""",1,603,45.0,301.0,16.0,579.278607
"""male""",0,203,45.0,349.0,32029.0,622.837438
"""male""",-1,161,45.0,321.0,-32774.0,610.10559
"""female""",1,586,45.0,297.0,16.0,582.049488
"""female""",0,188,45.0,332.0,31983.0,616.382979


Looking at the aggregations in the reference to the customers gender, we can see: 
1) The number of pages doesn't affect renting that much. People hold books with more pages longer however
2) Same can be said for the price of the books, higher the price tends not to return the book
3) Looking at the same labels and the numbers of renters, their ratio, it looks even.

### Potential features in relation to the label

In [31]:
education_agg_df = sub_library_df.group_by("education", "book_returned").agg(
    returning_book=pl.count("education")
)
education_agg_df = education_agg_df.drop_nulls()

education_agg_df = education_agg_df.filter(pl.col("book_returned") != -1)

education_agg_df.hvplot.table(
    columns=["education", "book_returned", "returning_book"],
    sortable=True,
    selectable=True,
)

:Table   [education,book_returned,returning_book]

In [32]:
occupation_agg_df = sub_library_df.group_by("occupation", "book_returned").agg(
    returning_book=pl.count("occupation")
)
occupation_agg_df = occupation_agg_df.drop_nulls()

occupation_agg_df = occupation_agg_df.filter(pl.col("book_returned") != -1)

occupation_agg_df.hvplot.table(
    columns=["occupation", "book_returned", "returning_book"],
    sortable=True,
    selectable=True,
)

:Table   [occupation,book_returned,returning_book]

In [33]:
age_agg_df = sub_library_df.group_by("age_category", "book_returned").agg(
    returning_book=pl.count("age_category")
)
age_agg_df = age_agg_df.drop_nulls()

age_agg_df = age_agg_df.filter(pl.col("book_returned") != -1)
age_agg_df = age_agg_df.filter(pl.col("age_category") != "Undefined")

age_agg_df.hvplot.table(
    columns=["age_category", "book_returned", "returning_book"],
    sortable=True,
    selectable=True,
)

:Table   [age_category,book_returned,returning_book]

Analyzing potential features such as education, occupation, age category with the label and without invalid data shows similar story, ~30% of the feature (by count) doesn't return book on time and this is the same for all features. 
There is no anomalies such as that one group is more dominant than the other.

This can mean that all feature will have the same improtance in determining the label, or none at all.

Interesting feature can be book `categories`, but because of the type (list of strings) and its nature (multi label) it can be more complex to analyze it. Intuitively, it can impact the prediction of the label (some categories may be more popular then the other) but I don't see it.

In [34]:
c_df = sub_library_df.group_by("categories", "book_returned").agg(
    returning_book=pl.count("categories")
)
c_df = c_df.drop_nulls()

c_df = c_df.filter(pl.col("book_returned") != -1)

c_df.hvplot.table(
    columns=["categories", "book_returned", "returning_book"],
    sortable=True,
    selectable=True,
)

:Table   [categories,book_returned,returning_book]

There are too many categories (147) and they apply the same principle and distribution ~ 70%:30% for returned and non-returned books.

### Finalizing dataset

After the feature analysis, final selection of the features is made. 

Features should be tested for correlation against the label. Another point to make is the labels are analyzed uniformly, without information about the libraries. Some libraries may have bigger issues than the others. 

In [35]:
dataset_df = library_df.select(
    [
        "patron_id",
        "id",
        "name",
        "gender",
        "education",
        "occupation",
        "age_category",
        "price",
        "pages",
        "book_returned",
    ]
).rename({"patron_id": "customer_id", "id": "book_id", "book_returned": "label"})

In [36]:
print(f"Dataset length before 'null' drops: {len(dataset_df)}\n")

dataset_df = dataset_df.drop_nulls()

print(f"Dataset length after 'null' drops: {len(dataset_df)}\n")

Dataset length before 'null' drops: 2000

Dataset length after 'null' drops: 1700



In [37]:
dataset_df = (
    dataset_df.with_columns(
        pl.col("gender").cast(pl.Categorical).to_physical().alias("gender")
    )
    .with_columns(
        pl.col("education").cast(pl.Categorical).to_physical().alias("education")
    )
    .with_columns(
        pl.col("occupation").cast(pl.Categorical).to_physical().alias("occupation")
    )
    .with_columns(
        pl.col("age_category").cast(pl.Categorical).to_physical().alias("age_category")
    )
)

In [38]:
dataset_df = dataset_df.with_columns(
    ((pl.col("price") - pl.col("price").mean()) / pl.col("price").std()).alias(
        "price_standardized"
    )
)

In [39]:
dataset_df = dataset_df.with_columns(
    ((pl.col("pages") - pl.col("pages").mean()) / pl.col("pages").std()).alias(
        "pages_standardized"
    )
)

In [40]:
# filter invalid labels
dataset_df = dataset_df.filter(pl.col("label") != -1)

#### Dataset visualized

In [41]:
dataset_df.hvplot.table(
    columns=[
        "gender",
        "education",
        "occupation",
        "age_category",
        "price_standardized",
        "pages_standardized",
        "label",
    ],
    sortable=True,
    selectable=True,
)

:Table   [gender,education,occupation,age_category,price_standardized,pages_standardized,label]

## Library EDA commentary

The goal of exploratory analysis is to provide insights and explain relationships between entities that are involved in the business process. By applying descriptive statistics techniques potential features are explored. 

## Motive 

The main motive or the business problem that drives the analysis is the late return of rented books from the libraries. Data exploration should provide an answer to whether this business problem can be translated into a statistical problem and whether can we mitigate it by collecting statistical data.

## Overall conclusion

The main entities that are presented and have enough information are `customers` and `books`. Information about libraries is not included in the analysis. The problem is presented uniformly and not by a particular library. 
After data cleaning (removing special characters, filling in missing values, standardizing formats, etc.) basic analysis is performed. For example, distribution of book pages and prices, and distribution of customer information such as gender, education, age group, etc. all concerning the label (is the book returned on time or not). 

The features are built that can potentially predict whether customers will return books on time or not.
Looking at the labels, there is a 70/30 ratio between positive and negative labels showing class imbalance (which makes training a model more difficult). This can be solved by using sampling techniques or weighting classes (or both).

There are no outstanding indicators in the data, for example, one particular feature stands out and shows that it is dominant for the outcome and the resulting label.

## The answers to the task

1) Building a model that can predict the likelihood of a late return of any book at the checkout time?

I would suggest the business (library) center their problem around the customers and explain that we can try to build a predictive model that can tell whether the customer will return a book or not based on their renting history. Having the checkout data, as well as the customer and book data, this is possible. Of course, this should be viewed as an experiment, as we can't know if the model results will satisfy the business requirements.

2) Are there any factors that are connected with the late returns?

Based on data analysis there are no dominant signs that some particular feature is connected with late returns. Books data shows that pricier books with more pages tend to be returned late with no specific user group returning books late (based on education, age group, etc.)

3) What would you recommend the library to do to mitigate the results you find?

30% of the books are returned late. Building a statistical model that can predict (on a satisfactory level) whether the customer will return a book is a good start. 
Even without the model, the library should have more insight and utilize the data collected to potentially build reports or dashboards, tracking the most popular books, number of rents per book, customer with the most rents, or similar **KPIs** which will help them get to the right business decision faster and with more confidence (backed by data).

4) How would you present your findings to get a buy-in?

The presentation can be in the form of a notebook or even better, in the form of a PowerPoint presentation. The notebooks are a mix of code, diagrams, and text which might not sit well with non-technical personnel.  
We should start with the description of the business problem, and how it affects the library business (they're losing money probably) and continue with a section that describes the findings in the data.
Throughout the presentation, we should aim to use non-technical terms and try to be concise and precise. 
When talking about building a model, set expectations and try to find out what is the minimal performance level that solves (or starts to) the already mentioned business problem. We shouldn't aim to build a model that has 95% accuracy (or some other metrics) performance, we should aim to build something that has impact on the business and that may be a model with weaker performance.  
The key takeaway is **managing expectations** which can be hard.